# Cost Allocation from Constraint Matrix

### Import packages

In [1]:
import pypsa
import pandas as pd
from helpers import get_linear_system, noisy_lopf
import networks

## Load example network

### One Bus, One Snapshot, Two Generators without invesment

$G_1 < d < G_1 + G_2$

Note, for such a system the total system cost $TC$ are less then the nodal payments as soon as one generator is at its limit: 

$\lambda = o_s + \bar{\mu_s} \;\;\;\;\; \forall s$

$d \, \lambda = d \, (o_s + \bar{\mu_s} ) \ge \sum_s g_s \, o_s = TC$

In [2]:
n = networks.n2_t9_g2_w()
noisy_lopf(n)

s = get_linear_system(n)
A_, A_inv, c, x, d, m, r = (s[k].round(1) for k in ['A_', 'A_inv', 'c', 'x', 'd', 'm', 'r'])

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
INFO:pypsa.linopf:Prepare linear problem
INFO:pypsa.linopf:Total preparation time: 0.15s
INFO:pypsa.linopf:Solve linear problem using Gurobi solver


Using license file /opt/gurobi900/gurobi.lic
Academic license - for non-commercial use only
Read LP format model from file /tmp/pypsa-problem-olr_ie7f.lp
Reading time = 0.00 seconds
obj: 78 rows, 31 columns, 150 nonzeros
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 78 rows, 31 columns and 150 nonzeros
Model fingerprint: 0x71c11937
Coefficient statistics:
  Matrix range     [2e-06, 1e+00]
  Objective range  [1e+00, 5e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 9e+01]
Presolve removed 42 rows and 19 columns
Presolve time: 0.01s
Presolved: 36 rows, 12 columns, 99 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8900021e+03   1.181248e+02   0.000000e+00      0s
       2    4.5897751e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  4.589775101e+03


INFO:pypsa.linopf:Optimization successful. Objective value: 4.59e+03


In [3]:
print(n.objective)
print(n.objective_constant)

4589.775100751555
0.0


In [4]:
n.loads_t.p_set

,0
0,50.0
1,49.0
2,48.0
3,47.0
4,46.0
5,90.0
6,46.5
7,47.5
8,48.5


In [5]:
n.generators_t.p.round(2)

,Gen0,Gen1
0,0.0,50.0
1,0.0,49.0
2,0.0,48.0
3,0.0,47.0
4,0.0,46.0
5,0.0,90.0
6,0.0,46.5
7,0.0,47.5
8,0.0,48.5


In [6]:
n.generators[['p_nom', 'p_nom_extendable', 'p_nom_opt', 'marginal_cost', 'capital_cost']].round(1)

,p_nom,p_nom_extendable,p_nom_opt,marginal_cost,capital_cost
Gen0,0.0,True,0.0,2.0,50.0
Gen1,0.0,True,90.0,4.0,10.0


# Understand the Cost Allocation 


Summerizing equations:

$\sum_i x_i \, A'_{i,j} = d_j \hspace{10pt} \leftrightarrow \hspace{10pt} x_i = \sum_j {A'}_{i,j}^{-1} \, d_j$

$\sum_j A'_{i,j} \, \mu_j = c_i \hspace{10pt} \leftrightarrow \hspace{10pt} \mu_j = \sum_i c_i \, {A'}_{i,j}^{-1}$

The total cost can be represented through all of the following expressions

$TC = \sum_i c_i \, x_i = \sum_{i,j}  c_i \, {A'}^{-1}_{ij} d_j = \sum_{i,j} x_i \, A'_{i,j} \, \mu_j = \sum_j \mu_j \, d_j $

In [7]:
assert all(A_.T @ x == d)

assert all(A_ @ m == c)

assert round(n.objective, 0) ==  A_inv @ d @ c == A_ @ m @ x

The basis of the cost allocation is ${A'}^{-1}$. It connects binding constraint to the variables

$x_i = \sum_j {A'}^{-1}_{i,j} \, d_j$

If ${A'}^{-1}_{i,j}$ is positive, $d_j$ has a positive effect on $x_i$. If negative, $d_j$ pushes $x_i$ down. 

In [8]:
A_inv * d

component                               Generator                          \
name                                 mu_nom_lower mu_upper                  
component_i                                  Gen0     Gen0                  
snapshot                                   static        0    1    2    3   
component name  component_i snapshot                                        
Generator p     Gen0        0                 0.0     -0.0  0.0  0.0  0.0   
                Gen1        0                -0.0      0.0  0.0  0.0  0.0   
                Gen0        1                 0.0      0.0 -0.0  0.0  0.0   
                Gen1        1                -0.0      0.0  0.0  0.0  0.0   
                Gen0        2                 0.0      0.0  0.0 -0.0  0.0   
                Gen1        2                -0.0      0.0  0.0  0.0  0.0   
                Gen0        3                 0.0      0.0  0.0  0.0 -0.0   
                Gen1        3                -0.0      0.0  0.0  0.0  0.0   
                Gen0        4                 0.0      0.0  0.0  0.0  0.0   
                Gen1        4                -0.0      0.0  0.0  0.0  0.0   
                Gen0        5                 0.0      0.0  0.0  0.0  0.0   
                Gen1        5                -0.0      0.0  0.0  0.0  0.0   
                Gen0        6                 0.0      0.0  0.0  0.0  0.0   
                Gen1        6                -0.0      0.0  0.0  0.0  0.0   
                Gen0        7                 0.0      0.0  0.0  0.0  0.0   
                Gen1        7                -0.0      0.0  0.0  0.0  0.0   
                Gen0        8                 0.0      0.0  0.0  0.0  0.0   
                Gen1        8                -0.0      0.0  0.0  0.0  0.0   
          p_nom Gen0        static            0.0      0.0  0.0  0.0  0.0   
                Gen1        static           -0.0      0.0  0.0  0.0  0.0   
Line      s_nom 1           static           -0.0      0.0  0.0  0.0  0.0   
          s     1           0                 0.0     -0.0  0.0  0.0  0.0   
                            1                 0.0      0.0 -0.0  0.0  0.0   
                            2                 0.0      0.0  0.0 -0.0  0.0   
                            3                 0.0      0.0  0.0  0.0 -0.0   
                            4                 0.0      0.0  0.0  0.0  0.0   
                            5                 0.0      0.0  0.0  0.0  0.0   
                            6                 0.0      0.0  0.0  0.0  0.0   
                            7                 0.0      0.0  0.0  0.0  0.0   
                            8                 0.0      0.0  0.0  0.0  0.0   

component                                                      ...  \
name                                                           ...   
component_i                                    Gen1 Gen0       ...   
snapshot                                4    5    5    6    7  ...   
component name  component_i snapshot                           ...   
Generator p     Gen0        0         0.0  0.0  0.0  0.0  0.0  ...   
                Gen1        0         0.0  0.0  0.0  0.0  0.0  ...   
                Gen0        1         0.0  0.0  0.0  0.0  0.0  ...   
                Gen1        1         0.0  0.0  0.0  0.0  0.0  ...   
                Gen0        2         0.0  0.0  0.0  0.0  0.0  ...   
                Gen1        2         0.0  0.0  0.0  0.0  0.0  ...   
                Gen0        3         0.0  0.0  0.0  0.0  0.0  ...   
                Gen1        3         0.0  0.0  0.0  0.0  0.0  ...   
                Gen0        4        -0.0 -0.0  0.0  0.0  0.0  ...   
                Gen1        4         0.0  0.0  0.0  0.0  0.0  ...   
                Gen0        5         0.0 -0.0  0.0  0.0  0.0  ...   
                Gen1        5         0.0  0.0  0.0  0.0  0.0  ...   
                Gen0        6         0.0  0.0  0.0 -0.0  0.0  ...   
                Gen1        6         0.0  0

The same counts for the cost of the variables $C_{i,j}$ defined as  

$C_{ij} = {A'}^{-1}_{ij} \, c_i \, d_j \hspace{10pt} \forall j \in \{j \, | \, d_j > 0\}$

If $C_{i,j}$ is positive, constraint $j$ pushes expences for variables $i$ up, if negative it lowers them. 

In [9]:
C = (r).mul(c, 0)
assert round((r.T @ c).sum(), 0) == round(n.objective, 0)
C

component                               Generator            Bus         \
name                                 mu_nom_lower marginal_price          
component_i                                  Gen0           Bus1          
snapshot                                   static              0      1   
component name  component_i snapshot                                      
Generator p     Gen0        0                 0.0            0.0    0.0   
                Gen1        0                -0.0          200.0    0.0   
                Gen0        1                 0.0            0.0    0.0   
                Gen1        1                -0.0            0.0  196.0   
                Gen0        2                 0.0            0.0    0.0   
                Gen1        2                -0.0            0.0    0.0   
                Gen0        3                 0.0            0.0    0.0   
                Gen1        3                -0.0            0.0    0.0   
                Gen0        4                 0.0            0.0    0.0   
                Gen1        4                -0.0            0.0    0.0   
                Gen0        5                 0.0            0.0    0.0   
                Gen1        5                -0.0            0.0    0.0   
                Gen0        6                 0.0            0.0    0.0   
                Gen1        6                -0.0            0.0    0.0   
                Gen0        7                 0.0            0.0    0.0   
                Gen1        7                -0.0            0.0    0.0   
                Gen0        8                 0.0            0.0    0.0   
                Gen1        8                -0.0            0.0    0.0   
          p_nom Gen0        static            0.0            0.0    0.0   
                Gen1        static           -0.0            0.0    0.0   
Line      s_nom 1           static           -0.0            0.0    0.0   
          s     1           0                 0.0           -0.0    0.0   
                            1                 0.0            0.0   -0.0   
                            2                 0.0            0.0    0.0   
                            3                 0.0            0.0    0.0   
                            4                 0.0            0.0    0.0   
                            5                 0.0            0.0    0.0   
                            6                 0.0            0.0    0.0   
                            7                 0.0            0.0    0.0   
                            8                 0.0            0.0    0.0   

component                                                                 \
name                                                                       
component_i                                                                
snapshot                                  2      3      4       5      6   
component name  component_i snapshot                                       
Generator p     Gen0        0           0.0    0.0    0.0     0.0    0.0   
                Gen1        0           0.0    0.0    0.0     0.0    0.0   
                Gen0        1           0.0    0.0    0.0     0.0    0.0   
                Gen1        1           0.0    0.0    0.0     0.0    0.0   
                Gen0        2           0.0    0.0    0.0     0.0    0.0   
                Gen1        2         192.0    0.0    0.0     0.0    0.0   
                Gen0        3           0.0    0.0    0.0     0.0    0.0   
                Gen1        3           0.0  188.0    0.0     0.0    0.0   
                Gen0        4           0.0    0.0    0.0     0.0    0.0   
                Gen1        4           0.0    0.0  184.0     0.0    0.0   
                Gen0        5           0.0    0.0    0.0     0.0    0.0   
                Gen1        5           0.0    0.0    0.0   360.0    0.0   
                Gen0        6           0.0    0.0    0.0     0.0    0.0   
      

**Here seems to be the problem:** 

$C_{i,j}$ has values below zero, see first two rows and have a look at column 3. Figuratively, this means that consumers at Bus1 and snapshot 3 receive money from Gen1, and redistribute it to Gen0. Since this happens also for other productions cost, it looses the uniqueness of the peer-to-peer payment. More concrete: We cannot say what amount consumers at Bus1 and snapshot 0 effectively pay to Gen0 at snapshot 0, since Gen0 is not payed until snapshot 3. 

In [10]:
C.sum().to_frame('Constraint induced cost')

Constraint induced cost
component name           component_i snapshot                         
Generator mu_nom_lower   Gen0        static                        0.0
Bus       marginal_price Bus1        0                           200.0
                                     1                           196.0
                                     2                           192.0
                                     3                           188.0
                                     4                           184.0
                                     5                          3060.0
                                     6                           186.0
                                     7                           190.0
                                     8                           194.0

In [11]:
C.sum(1).to_frame('Cost per variable')

Cost per variable
component name  component_i snapshot                   
Generator p     Gen0        0                       0.0
                Gen1        0                     200.0
                Gen0        1                       0.0
                Gen1        1                     196.0
                Gen0        2                       0.0
                Gen1        2                     192.0
                Gen0        3                       0.0
                Gen1        3                     188.0
                Gen0        4                       0.0
                Gen1        4                     184.0
                Gen0        5                       0.0
                Gen1        5                     360.0
                Gen0        6                       0.0
                Gen1        6                     186.0
                Gen0        7                       0.0
                Gen1        7                     190.0
                Gen0        8                       0.0
                Gen1        8                     194.0
          p_nom Gen0        static                  0.0
                Gen1        static                900.0
Line      s_nom 1           static               1800.0
          s     1           0                       0.0
                            1                       0.0
                            2                       0.0
                            3                       0.0
                            4                       0.0
                            5                       0.0
                            6                       0.0
                            7                       0.0
                            8                       0.0

### Other direction

The matrix $A'_{i,j}$ connects to the variables their binding constraints

$\sum_i x_i \, A'_{i,j} = d_j$


In [12]:
A_.mul(x, 0)

component                               Generator                          \
name                                 mu_nom_lower mu_upper                  
component_i                                  Gen0     Gen0                  
snapshot                                   static        0    1    2    3   
component name  component_i snapshot                                        
Generator p     Gen0        0                 0.0     -0.0  0.0  0.0  0.0   
                Gen1        0                 0.0      0.0  0.0  0.0  0.0   
                Gen0        1                 0.0      0.0 -0.0  0.0  0.0   
                Gen1        1                 0.0      0.0  0.0  0.0  0.0   
                Gen0        2                 0.0      0.0  0.0 -0.0  0.0   
                Gen1        2                 0.0      0.0  0.0  0.0  0.0   
                Gen0        3                 0.0      0.0  0.0  0.0 -0.0   
                Gen1        3                 0.0      0.0  0.0  0.0  0.0   
                Gen0        4                 0.0      0.0  0.0  0.0  0.0   
                Gen1        4                 0.0      0.0  0.0  0.0  0.0   
                Gen0        5                 0.0      0.0  0.0  0.0  0.0   
                Gen1        5                 0.0      0.0  0.0  0.0  0.0   
                Gen0        6                 0.0      0.0  0.0  0.0  0.0   
                Gen1        6                 0.0      0.0  0.0  0.0  0.0   
                Gen0        7                 0.0      0.0  0.0  0.0  0.0   
                Gen1        7                 0.0      0.0  0.0  0.0  0.0   
                Gen0        8                 0.0      0.0  0.0  0.0  0.0   
                Gen1        8                 0.0      0.0  0.0  0.0  0.0   
          p_nom Gen0        static            0.0      0.0  0.0  0.0  0.0   
                Gen1        static            0.0      0.0  0.0  0.0  0.0   
Line      s_nom 1           static            0.0      0.0  0.0  0.0  0.0   
          s     1           0                -0.0     -0.0 -0.0 -0.0 -0.0   
                            1                -0.0     -0.0 -0.0 -0.0 -0.0   
                            2                -0.0     -0.0 -0.0 -0.0 -0.0   
                            3                -0.0     -0.0 -0.0 -0.0 -0.0   
                            4                -0.0     -0.0 -0.0 -0.0 -0.0   
                            5                -0.0     -0.0 -0.0 -0.0 -0.0   
                            6                -0.0     -0.0 -0.0 -0.0 -0.0   
                            7                -0.0     -0.0 -0.0 -0.0 -0.0   
                            8                -0.0     -0.0 -0.0 -0.0 -0.0   

component                                                       ...  \
name                                                            ...   
component_i                                     Gen1 Gen0       ...   
snapshot                                4    5     5    6    7  ...   
component name  component_i snapshot                            ...   
Generator p     Gen0        0         0.0  0.0   0.0  0.0  0.0  ...   
                Gen1        0         0.0  0.0   0.0  0.0  0.0  ...   
                Gen0        1         0.0  0.0   0.0  0.0  0.0  ...   
                Gen1        1         0.0  0.0   0.0  0.0  0.0  ...   
                Gen0        2         0.0  0.0   0.0  0.0  0.0  ...   
                Gen1        2         0.0  0.0   0.0  0.0  0.0  ...   
                Gen0        3         0.0  0.0   0.0  0.0  0.0  ...   
                Gen1        3         0.0  0.0   0.0  0.0  0.0  ...   
                Gen0        4        -0.0  0.0   0.0  0.0  0.0  ...   
                Gen1        4         0.0  0.0   0.0  0.0  0.0  ...   
                Gen0        5         0.0 -0.0   0.0  0.0  0.0  ...   
                Gen1        5         0.0  0.0 -90.0  0.0  0.0  ...   
                Gen0        6         0.0  0.0   0.0 -0.0  0.0  ...   
                Gen1      

In [13]:
D = (A_ * m).mul(x, 0) 
assert round(n.objective, 0) ==  D.sum().sum()
D

component                               Generator                          \
name                                 mu_nom_lower mu_upper                  
component_i                                  Gen0     Gen0                  
snapshot                                   static        0    1    2    3   
component name  component_i snapshot                                        
Generator p     Gen0        0                 0.0     -0.0  0.0  0.0  0.0   
                Gen1        0                 0.0      0.0  0.0  0.0  0.0   
                Gen0        1                 0.0      0.0 -0.0  0.0  0.0   
                Gen1        1                 0.0      0.0  0.0  0.0  0.0   
                Gen0        2                 0.0      0.0  0.0 -0.0  0.0   
                Gen1        2                 0.0      0.0  0.0  0.0  0.0   
                Gen0        3                 0.0      0.0  0.0  0.0 -0.0   
                Gen1        3                 0.0      0.0  0.0  0.0  0.0   
                Gen0        4                 0.0      0.0  0.0  0.0  0.0   
                Gen1        4                 0.0      0.0  0.0  0.0  0.0   
                Gen0        5                 0.0      0.0  0.0  0.0  0.0   
                Gen1        5                 0.0      0.0  0.0  0.0  0.0   
                Gen0        6                 0.0      0.0  0.0  0.0  0.0   
                Gen1        6                 0.0      0.0  0.0  0.0  0.0   
                Gen0        7                 0.0      0.0  0.0  0.0  0.0   
                Gen1        7                 0.0      0.0  0.0  0.0  0.0   
                Gen0        8                 0.0      0.0  0.0  0.0  0.0   
                Gen1        8                 0.0      0.0  0.0  0.0  0.0   
          p_nom Gen0        static            0.0      0.0  0.0  0.0  0.0   
                Gen1        static            0.0      0.0  0.0  0.0  0.0   
Line      s_nom 1           static            0.0      0.0  0.0  0.0  0.0   
          s     1           0                -0.0     -0.0 -0.0 -0.0 -0.0   
                            1                -0.0     -0.0 -0.0 -0.0 -0.0   
                            2                -0.0     -0.0 -0.0 -0.0 -0.0   
                            3                -0.0     -0.0 -0.0 -0.0 -0.0   
                            4                -0.0     -0.0 -0.0 -0.0 -0.0   
                            5                -0.0     -0.0 -0.0 -0.0 -0.0   
                            6                -0.0     -0.0 -0.0 -0.0 -0.0   
                            7                -0.0     -0.0 -0.0 -0.0 -0.0   
                            8                -0.0     -0.0 -0.0 -0.0 -0.0   

component                                                        ...  \
name                                                             ...   
component_i                                      Gen1 Gen0       ...   
snapshot                                4    5      5    6    7  ...   
component name  component_i snapshot                             ...   
Generator p     Gen0        0         0.0  0.0    0.0  0.0  0.0  ...   
                Gen1        0         0.0  0.0    0.0  0.0  0.0  ...   
                Gen0        1         0.0  0.0    0.0  0.0  0.0  ...   
                Gen1        1         0.0  0.0    0.0  0.0  0.0  ...   
                Gen0        2         0.0  0.0    0.0  0.0  0.0  ...   
                Gen1        2         0.0  0.0    0.0  0.0  0.0  ...   
                Gen0        3         0.0  0.0    0.0  0.0  0.0  ...   
                Gen1        3         0.0  0.0    0.0  0.0  0.0  ...   
                Gen0        4        -0.0  0.0    0.0  0.0  0.0  ...   
                Gen1        4         0.0  0.0    0.0  0.0  0.0  ...   
                Gen0        5         0.0 -0.0    0.0  0.0  0.0  ...   
                Gen1        5         0.0  0.0 -900.0  0.0  0.0  ...   
                Gen0        6         0.0  0.0    0.0 -0.0  0.0  ...   
        